In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import Runnable
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

class CustomRunnable(Runnable):
    def invoke(self, inputs, config=None):
        question = inputs["question"]
        context = "\n".join([doc.page_content for doc in retriver.get_relevant_documents(question)])  # 간결하게 context 생성
        chat_history = memory.load_memory_variables({})["chat_history"]
        chat_history_str = "\n".join([message["text"] for message in chat_history]) if isinstance(chat_history, list) else chat_history
        return {"context": chat_history_str + "\n" + context, "question": question}

custom_runnable = CustomRunnable()

chain = (
    custom_runnable
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result.content)  # AI의 답변만 출력
    return result

# 연속적으로 4개의 질문을 호출
invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")
invoke_chain("What was the first question that I asked?")


Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.


TypeError: 'HumanMessage' object is not subscriptable